In [1]:
from sqlalchemy import create_engine
engine = create_engine("sqlite+pysqlite:///database.sqlite", echo=True)

In [2]:
from sqlalchemy import String, ForeignKey
# 建立声明性基础
from sqlalchemy.orm import DeclarativeBase
class Base(DeclarativeBase):
    pass

# 声明映射类
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = "user_account"
    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]
    addresses: Mapped[List["Address"]] = relationship(back_populates="user")
    def __repr__(self) -> str:
        return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"
    
class Address(Base):
    __tablename__ = "address"
    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id = mapped_column(ForeignKey("user_account.id"))
    user: Mapped[User] = relationship(back_populates="addresses")
    def __repr__(self) -> str:
        return f"Address(id={self.id!r}, email_address={self.email_address!r})"

### 类的实例代表行

In [3]:
squidward = User(name="squidward", fullname="Squidward Tentacles")
krabs = User(name="ehkrabs", fullname="Eugene H. Krabs")

In [4]:
squidward

User(id=None, name='squidward', fullname='Squidward Tentacles')

### 将对象添加到会话

为了逐步说明添加过程，我们将创建一个 不使用上下文管理器的Session （因此我们必须确保稍后关闭它！）：

In [5]:
from sqlalchemy.orm import Session

session = Session(engine)

In [6]:
# 然后使用以下方法将对象添加到Session中 Session.add()方法。调用此方法时，对象处于挂起状态，尚未插入：
session.add(squidward)
session.add(krabs)

In [7]:
# 当我们有挂起的对象时，我们可以通过查看Session上名为Session.new集合来查看此状态：
session.new
# 上面的视图使用了一个名为IdentitySet的集合，它本质上是一个 Python 集合，在所有情况下都会对对象身份进行哈希处理（即使用 Python 内置id()函数，而不是 Python hash()函数）。

IdentitySet([User(id=None, name='squidward', fullname='Squidward Tentacles'), User(id=None, name='ehkrabs', fullname='Eugene H. Krabs')])

此时还未写入数据库

上面我们观察到Session首先被调用来发出 SQL，因此它创建了一个新事务并为两个对象发出了适当的 INSERT 语句。事务现在保持打开状态，直到我们调用Session.commit() 、 Session.rollback()或 Session的Session.close()方法。

虽然Session.flush()可用于手动推出待处理的 对当前事务的更改，通常是不必要的，因为 Session具有称为autoflush 的行为，该行为 我们稍后会举例说明。它也会在任何时候刷新更改 调用Session.commit() 。

In [8]:
session.flush()

2024-12-25 16:28:08,927 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-25 16:28:08,928 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-12-25 16:28:08,929 INFO sqlalchemy.engine.Engine [generated in 0.00007s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('squidward', 'Squidward Tentacles')
2024-12-25 16:28:08,930 INFO sqlalchemy.engine.Engine INSERT INTO user_account (name, fullname) VALUES (?, ?) RETURNING id
2024-12-25 16:28:08,931 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('ehkrabs', 'Eugene H. Krabs')


In [ ]:
# session.commit()

In [9]:
squidward

User(id=1, name='squidward', fullname='Squidward Tentacles')

In [10]:
krabs

User(id=2, name='ehkrabs', fullname='Eugene H. Krabs')

In [11]:
some_squidward = session.get(User, 1)

In [12]:
some_squidward

User(id=1, name='squidward', fullname='Squidward Tentacles')

In [13]:
some_squidward is squidward

True

### committing 提交

In [15]:
session.commit()

2024-12-25 16:28:40,264 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
session.new

IdentitySet([])

In [17]:
from sqlalchemy import select

sandy = session.execute(select(User).filter_by(name="squidward")).scalar_one()

2024-12-25 16:28:44,932 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-12-25 16:28:44,933 INFO sqlalchemy.engine.Engine [generated in 0.00075s] ('squidward',)


In [18]:
sandy

User(id=1, name='squidward', fullname='Squidward Tentacles')

In [19]:
sandy.fullname = "Sandy Squirrel"

In [20]:
# 该对象出现在名为Session.dirty的集合中，表明该对象是“脏的”：
sandy in session.dirty

True

In [21]:
sandy_fullname = session.execute(select(User.fullname).where(User.id == 1)).scalar_one()

2024-12-25 16:28:51,229 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.id = ?
2024-12-25 16:28:51,229 INFO sqlalchemy.engine.Engine [generated in 0.00073s] ('Sandy Squirrel', 1)
2024-12-25 16:28:51,231 INFO sqlalchemy.engine.Engine SELECT user_account.fullname 
FROM user_account 
WHERE user_account.id = ?
2024-12-25 16:28:51,232 INFO sqlalchemy.engine.Engine [generated in 0.00052s] (1,)


In [22]:
sandy_fullname

'Sandy Squirrel'

In [23]:
sandy in session.dirty

False

### 使用工作单元模式删除 ORM 对象

In [25]:
ehkrabs = session.get(User, 2)

In [26]:
session.delete(ehkrabs)

In [27]:
session.execute(select(User).where(User.name == "ehkrabs")).first()

2024-12-25 16:30:48,511 INFO sqlalchemy.engine.Engine SELECT address.id AS address_id, address.email_address AS address_email_address, address.user_id AS address_user_id 
FROM address 
WHERE ? = address.user_id
2024-12-25 16:30:48,512 INFO sqlalchemy.engine.Engine [generated in 0.00098s] (2,)
2024-12-25 16:30:48,514 INFO sqlalchemy.engine.Engine UPDATE address SET user_id=? WHERE address.id = ?
2024-12-25 16:30:48,516 INFO sqlalchemy.engine.Engine [generated in 0.00102s] [(None, 2), (None, 3), (None, 5), (None, 6)]
2024-12-25 16:30:48,518 INFO sqlalchemy.engine.Engine DELETE FROM user_account WHERE user_account.id = ?
2024-12-25 16:30:48,519 INFO sqlalchemy.engine.Engine [generated in 0.00092s] (2,)
2024-12-25 16:30:48,520 INFO sqlalchemy.engine.Engine SELECT user_account.id, user_account.name, user_account.fullname 
FROM user_account 
WHERE user_account.name = ?
2024-12-25 16:30:48,520 INFO sqlalchemy.engine.Engine [cached since 123.6s ago] ('ehkrabs',)


In [28]:
ehkrabs in session

False

批量/多行插入、更新插入、更新和删除

In [29]:
session.rollback()

2024-12-25 16:34:33,957 INFO sqlalchemy.engine.Engine ROLLBACK


In [35]:
sandy.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1f30c416980>,
 'fullname': 'Squidward Tentacles',
 'name': 'squidward',
 'id': 1}

In [42]:
sandy.fullname

'Squidward Tentacles'

In [43]:
ehkrabs in session

True

In [0]:
result = session.execute(select(User).where(User.name == "ehkrabs")).scalar_one()

In [47]:
result

User(id=2, name='ehkrabs', fullname='Eugene H. Krabs')

In [50]:
ehkrabs is result

True

In [70]:
session.close()

关闭会话
- 它将所有连接资源释放到连接池，取消（例如回滚）任何正在进行的事务。

- 这意味着当我们利用会话执行一些只读操作时 任务然后关闭它，我们不需要显式调用 Session.rollback()确保事务回滚；连接池处理这个。

- 它从Session中删除所有对象。

- 这意味着我们为此Session加载的所有 Python 对象（如sandy 、 patrick和squidward ）现在都处于称为detached 的状态。特别是，我们会注意到，仍然处于过期状态的对象（例如由于调用Session.commit()现在不起作用，因为它们不包含当前行的状态，并且不具有任何功能。不再与要刷新的任何数据库事务关联：